#**스마트폰 센서 데이터 기반 모션 분류**
# 단계3 : 단계별 모델링


## 0.미션

단계별로 나눠서 모델링을 수행하고자 합니다.  

* 단계1 : 정적(0), 동적(1) 행동 분류 모델 생성
* 단계2 : 세부 동작에 대한 분류모델 생성
    * 단계1 모델에서 0으로 예측 -> 정적 행동 3가지 분류 모델링
    * 단계1 모델에서 1으로 예측 -> 동적 행동 3가지 분류 모델링
* 모델 통합
    * 두 단계 모델을 통합하고, 새로운 데이터에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
* 성능 비교
    * 기본 모델링의 성능과 비교
    * 모든 모델링은 [다양한 알고리즘 + 성능 튜닝]을 수행해야 합니다.


## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.





### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용

 <br/>  

* 세부 요구사항
    - data01_train.csv 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - data01_test.csv 를 불러와 'new_data' 이름으로 저장합니다.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/미니프로젝트/5차_/2023.10.25_미니프로젝트5차_실습자료, 데이터/데이터/data01_train.csv')

In [ ]:
new_data = pd.read_csv('/content/drive/MyDrive/미니프로젝트/5차_/2023.10.25_미니프로젝트5차_실습자료, 데이터/데이터/data01_test.csv')

In [ ]:
x_test= pd.read_csv('/content/drive/MyDrive/미니프로젝트/5차_/2023.10.25_미니프로젝트5차_실습자료, 데이터/데이터/data01_test.csv')

## 2.데이터 전처리

* 세부 요구사항
    - Label 추가 : data 에 Activity_dynamic 를 추가합니다. Activity_dynamic은 과제1에서 is_dynamic과 동일한 값입니다.
    - x와 y1, y2로 분할하시오.
        * y1 : Activity
        * y2 : Activity_dynamic
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [ ]:
is_dynamic = {
    'STANDING' : 0,
    'SITTING' : 0,
    'LAYING' : 0,
    'WALKING' : 1,
    'WALKING_UPSTAIRS' : 1,
    'WALKING_DOWNSTAIRS' : 1
}
data['Activity_dynamic'] = data['Activity'].replace(is_dynamic)



In [ ]:
x = data.drop(columns=['Activity', 'Activity_dynamic'])
y1 = data['Activity']
y2 = data['Activity_dynamic']


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y1, test_size=0.2, random_state=42, stratify=y1)
x_train_d, x_val_d, y_train_d, y_val_d = train_test_split(x, y2, test_size=0.2, random_state=42, stratify=y2)
x_val.shape,x_val_d.shape

((1177, 562), (1177, 562))

## **3.단계별 모델링**

![](https://github.com/DA4BAM/image/blob/main/step%20by%20step.png?raw=true)

### (1) 단계1 : 정적/동적 행동 분류 모델

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)과 동적 행동(동적 : Walking, Walking-Up, Walking-Down)을 구분하는 모델 생성.
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

#### 1) 알고리즘1 :

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier()
rf1.fit(x_train, y_train)

rf1.score(x_val,y_val)

0.9762107051826678

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf2 = RandomForestClassifier()
rf2.fit(x_train_d, y_train_d)

rf2.score(x_val_d,y_val_d)

0.9991503823279524

#### 2) 알고리즘2 :

In [ ]:
from lightgbm import LGBMClassifier

lgb=LGBMClassifier(random_state=42)
lgb.fit(x_train.values,y_train.values)

lgb.score(x_val,y_val)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.044440 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 140168
[LightGBM] [Info] Number of data points in the train set: 4704, number of used features: 562
[LightGBM] [Info] Start training from score -1.662702
[LightGBM] [Info] Start training from score -1.739574
[LightGBM] [Info] Start training from score -1.688825
[LightGBM] [Info] Start training from score -1.774060
[LightGBM] [Info] Start training from score -2.005698
[LightGBM] [Info] Start training from score -1.925291
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightG

0.9915038232795242

In [ ]:
from lightgbm import LGBMClassifier

lgb=LGBMClassifier(random_state=42)
lgb.fit(x_train_d.values,y_train_d.values)

lgb.score(x_val_d,y_val_d)

[LightGBM] [Info] Number of positive: 2117, number of negative: 2587
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.042315 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 140167
[LightGBM] [Info] Number of data points in the train set: 4704, number of used features: 562
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.450043 -> initscore=-0.200499
[LightGBM] [Info] Start training from score -0.200499
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

0.9991503823279524

### (2) 단계2-1 : 정적 동작 세부 분류

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)인 데이터 추출
    * Laying, Sitting, Standing 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [ ]:
s_data = data[data['Activity_dynamic'] == 0].reset_index()
s_data['Activity'].value_counts()

LAYING      1115
STANDING    1087
SITTING     1032
Name: Activity, dtype: int64

In [ ]:
s_data

,index,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,Activity_dynamic
0,0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,...,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163,21,STANDING,0
1,1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,...,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944,15,LAYING,0
2,2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,...,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755,11,STANDING,0
3,7,0.272026,-0.001329,-0.125491,-0.992068,-0.912985,-0.972451,-0.994752,-0.943141,-0.976428,...,-0.024442,0.076332,0.741277,0.729812,-0.817201,0.037746,0.136129,16,STANDING,0
4,8,0.284338,0.021956,-0.006925,-0.980153,-0.838394,-0.782357,-0.983683,-0.816199,-0.743923,...,0.021212,-0.009465,-0.282762,0.563343,-0.782072,0.242834,-0.025285,22,STANDING,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3229,5874,0.257476,-0.413865,0.017374,-0.919666,0.075259,-0.630716,-0.937986,0.176796,-0.613510,...,-0.000793,0.267189,-0.186202,0.098099,0.792970,-0.034020,-0.928148,1,LAYING,0
3230,5875,0.277378,-0.013298,-0.104322,-0.996596,-0.987491,-0.973345,-0.996372,-0.987746,-0.973512,...,0.122320,0.136275,-0.708377,-0.507788,-0.818263,0.222620,0.035430,27,STANDING,0
3231,5876,0.277194,-0.012389,-0.131974,-0.994046,-0.940578,-0.917337,-0.994261,-0.932830,-0.908088,...,-0.034888,-0.261437,-0.391477,-0.877612,-0.912365,0.114009,0.080146,21,SITTING,0
3232,5878,0.267981,-0.018348,-0.107440,-0.991303,-0.989881,-0.990313,-0.992386,-0.988852,-0.991237,...,0.060173,0.228739,0.684400,-0.216665,0.620363,-0.437247,-0.571840,19,LAYING,0


In [ ]:
sx=s_data.drop(columns=['Activity','Activity_dynamic'])
sy=s_data['Activity']

In [ ]:
from sklearn.model_selection import train_test_split
x_train_s, x_val_s,y_train_s,y_val_s = train_test_split(sx,sy,test_size=0.2, random_state=42, stratify=sy)

In [ ]:
s_rf = RandomForestClassifier()
s_rf.fit(x_train_s, y_train_s)
s_rf.score(x_val_s, y_val_s)

0.9783616692426584

In [ ]:
from lightgbm import LGBMClassifier

lgb=LGBMClassifier(random_state=42)
lgb.fit(x_train_s.values,y_train_s.values)

lgb.score(x_val_s,y_val_s)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.074435 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 139634
[LightGBM] [Info] Number of data points in the train set: 2587, number of used features: 563
[LightGBM] [Info] Start training from score -1.064788
[LightGBM] [Info] Start training from score -1.141659
[LightGBM] [Info] Start training from score -1.090911
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning

0.9876352395672334

### (3) 단계2-2 : 동적 동작 세부 분류

* 세부 요구사항
    * 동동적 행동(Walking, Walking Upstairs, Walking Downstairs)인 데이터 추출
    * Walking, Walking Upstairs, Walking Downstairs 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [ ]:
d_data = data[data['Activity_dynamic'] == 1].reset_index()
d_data['Activity'].value_counts()

WALKING               998
WALKING_UPSTAIRS      858
WALKING_DOWNSTAIRS    791
Name: Activity, dtype: int64

In [ ]:
dx=d_data.drop(columns=['Activity','Activity_dynamic'])
dy=d_data['Activity']

In [ ]:
x_train_d, x_val_d,y_train_d,y_val_d = train_test_split(dx,dy,test_size=0.2, random_state=42, stratify=dy)

In [ ]:
d_rf = RandomForestClassifier()
d_rf.fit(x_train_d, y_train_d)
d_rf.score(x_val_d, y_val_d)

0.9811320754716981

In [ ]:
from lightgbm import LGBMClassifier

lgb=LGBMClassifier(random_state=42)
lgb.fit(x_train_d.values,y_train_d.values)

lgb.score(x_val_d,y_val_d)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027657 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 140063
[LightGBM] [Info] Number of data points in the train set: 2117, number of used features: 563
[LightGBM] [Info] Start training from score -0.975647
[LightGBM] [Info] Start training from score -1.207285
[LightGBM] [Info] Start training from score -1.126878
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning

0.9962264150943396

### (4) 분류 모델 합치기


* 세부 요구사항
    * 두 단계 모델을 통합하고, 새로운 데이터(test)에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
    * 데이터 파이프라인 구축 : test데이터가 로딩되어 전처리 과정을 거치고, 예측 및 성능 평가 수행

![](https://github.com/DA4BAM/image/blob/main/pipeline%20function.png?raw=true)

#### 1) 함수 만들기

In [ ]:
temp_data = data.drop(['Activity', 'Activity_dynamic'], axis=1)
# 정적, 동적 예측 결과 저장
temp_data['is_dynamic'] = rf2.predict(temp_data)

In [ ]:
# 정적과 동적 예측 결과에 따라 나누기
# reset_index로 원래의 index 를 column으로 빼서 나중에 concat 후 사용
temp_static = temp_data[temp_data['is_dynamic'] == 0].reset_index()
temp_dynamic = temp_data[temp_data['is_dynamic'] == 1].reset_index()